此notebook用于检测获取数据的准确性

In [1]:
from qmtbt import QMTStore
from datetime import datetime
import backtrader as bt
import pandas as pd
import efinance
import matplotlib.pyplot as plt

In [33]:
from xtquant.xtdata import download_history_data2
#coding=utf-8

import pandas as pd
import sys
sys.path.append(r'E:\\software\\QMT\\userdata_mini')

# 加载xtquant库
from xtquant.xtdata import *

def on_progress(data):
    '''补充历史数据回调函数'''
    print(data) 
    
if __name__ == "__main__":
    print("xtdata demo")
    # 根据stock_list下载数据
    stock_list = ['603909.SH','300450.SZ','600740.SH','002362.SZ','600750.SH']
    field_list = ['time','open','close','low','high','volume'] # 提取的字段
    download_history_data2(stock_list, period='1d', start_time='20230201', end_time='20230223', callback=on_progress)
    print('download_history_data2 finished')
    
    # 获取股票close数据
    ret = get_market_data(field_list, stock_list, period='1d', start_time='', end_time='', count=5, dividend_type='front', fill_data=True)
    print(ret['close'].T)


xtdata demo
download_history_data2 finished
{'finished': 1, 'total': 5, 'stockcode': '', 'message': '603909.SH'}
          603909.SH  300450.SZ  600740.SH  002362.SZ  600750.SH
20241122       9.15     22.627       4.48      14.96      21.57
20241125       9.39     22.627       4.48      14.96      21.57
20241126       9.27     22.627       4.48      14.96      21.57
20241127       9.34     22.627       4.48      14.96      21.57
20241128       9.44     22.627       4.48      14.96      21.57


In [4]:
import backtrader as bt
import pandas as pd

# 定义一个简单的策略，用于记录每条 K 线的数据
class PrintData(bt.Strategy):
    def __init__(self):
        pass

    def next(self):
        # 获取当前的 K 线数据
        current_data = self.datas[0]
        
        # 提取当前 K 线的数据
        date = current_data.datetime.date(0)  # 获取日期
        open_price = current_data.open[0]     # 获取开盘价
        high_price = current_data.high[0]     # 获取最高价
        low_price = current_data.low[0]       # 获取最低价
        close_price = current_data.close[0]   # 获取收盘价
        volume = current_data.volume[0]       # 获取成交量
        
        # 输出数据
        print(f"Date: {date}, Open: {open_price}, High: {high_price}, Low: {low_price}, Close: {close_price}, Volume: {volume}")

# 初始化 cerebro 引擎
cerebro = bt.Cerebro()

# 加载数据
store = QMTStore()
data = store.getdata(dataname='000001.SZ', timeframe=bt.TimeFrame.Days, fromdate=datetime(2020, 1, 1),
                      todate=datetime(2021, 1, 1), live=False,dividend_type='none', fill_data=False)

# 将数据添加到 cerebro
cerebro.adddata(data)

# 添加策略
cerebro.addstrategy(PrintData)

# 运行回测
cerebro.run()
cerebro.plot()


下载数据000001.SZ
000001.SZ历史数据装载成功！
1
Date: 2020-01-02, Open: 14.774000000000001, High: 15.073999999999998, Low: 14.674000000000001, Close: 14.994, Volume: 1530231.0
Date: 2020-01-03, Open: 15.064, High: 15.434000000000001, Low: 15.044, Close: 15.304000000000002, Volume: 1116194.0
Date: 2020-01-06, Open: 15.134, High: 15.463999999999999, Low: 15.033999999999999, Close: 15.193999999999999, Volume: 862083.0
Date: 2020-01-07, Open: 15.253999999999998, High: 15.403999999999996, Low: 15.073999999999998, Close: 15.273999999999997, Volume: 728607.0
Date: 2020-01-08, Open: 15.123999999999999, High: 15.174, Low: 14.753999999999998, Close: 14.783999999999999, Volume: 847824.0
Date: 2020-01-09, Open: 14.934000000000001, High: 15.053999999999998, Low: 14.654000000000002, Close: 14.914000000000001, Volume: 1031636.0
Date: 2020-01-10, Open: 14.913999999999998, High: 14.933999999999997, Low: 14.644, Close: 14.814, Volume: 585548.0
Date: 2020-01-13, Open: 14.873999999999999, High: 15.154, Low: 14.734, Cl

<IPython.core.display.Javascript object>

C:\Users\21009\AppData\Local\Programs\Python\Python39\lib\site-packages\matplotlib\backends\backend_nbagg.py:181: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm('matplotlib', data={'id': self.uuid})


[[<Figure size 640x480 with 4 Axes>]]

In [5]:

def get_k_data(stock_code, begin: datetime, end: datetime):
    """根据efinance工具包获取股票数据
    :param stock_code:股票代码
    :param begin: 开始日期
    :param end: 结束日期
    """
    # stock_code = '600519'  # 股票代码，茅台
    k_dataframe: pd.DataFrame = efinance.stock.get_quote_history(
        stock_code, beg=begin.strftime("%Y%m%d"), end=end.strftime("%Y%m%d"))
    k_dataframe = k_dataframe.iloc[:, :9]
    k_dataframe.columns = ['name', 'code', 'date', 'open', 'close', 'high', 'low', 'volume', 'turnover']
    k_dataframe.index = pd.to_datetime(k_dataframe.date)
    k_dataframe.drop(['name', 'code', "date"], axis=1, inplace=True)
    return k_dataframe

start_time = datetime(2020, 1, 1)
end_time = datetime(2021, 1, 1)
cerebro = bt.Cerebro()

dataframe = get_k_data('000001', begin=start_time, end=end_time)
data = bt.feeds.PandasData(dataname=dataframe, fromdate=start_time, todate=end_time)

# 将数据添加到cerebro引擎
cerebro.adddata(data)

# 运行策略（这里没有添加策略，只是为了展示绘图功能）
cerebro.run()

# 绘制图表
cerebro.plot()


<IPython.core.display.Javascript object>

C:\Users\21009\AppData\Local\Programs\Python\Python39\lib\site-packages\matplotlib\backends\backend_nbagg.py:181: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm('matplotlib', data={'id': self.uuid})


[[<Figure size 640x480 with 4 Axes>]]